# Introduction to Reference Frames in Computer Vision

Simple example of a reference frame and a point in space.

In [1]:
import numpy as np
import plotly.graph_objects as go

# Create a 2D point
point = np.array([1, 1])

# Create a figure with Plotly
fig = go.Figure()

# Add the reference frame axes (origin)
fig.add_trace(go.Scatter(
    x=[-2, 2], y=[0, 0],
    mode='lines',
    line=dict(color='red', width=2),
    name='X Axis'
))

fig.add_trace(go.Scatter(
    x=[0, 0], y=[-2, 2],
    mode='lines',
    line=dict(color='green', width=2),
    name='Y Axis'
))

# Add the origin
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    mode='markers',
    marker=dict(color='black', size=8, symbol='circle'),
    name='Origin'
))

# Add the point
fig.add_trace(go.Scatter(
    x=[point[0]], y=[point[1]],
    mode='markers+text',
    marker=dict(color='blue', size=10),
    text=['P'],
    textposition="top center",
    name=f"Point P ({point[0]}, {point[1]})"
))

# Draw a dashed line from the origin to the point
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='blue', width=1, dash='dash'),
    name='Connection to frame'
))

# Add projections on the axes as dotted lines
# Projection on X axis
fig.add_trace(go.Scatter(
    x=[0, point[0]], y=[0, 0],
    mode='lines+text',
    line=dict(color='blue', width=1, dash='dot'),
    text=["", f"x={point[0]}"],
    textposition="bottom center",
    name='X Projection'
))

# Projection on Y axis
fig.add_trace(go.Scatter(
    x=[0, 0], y=[0, point[1]],
    mode='lines+text',
    line=dict(color='blue', width=1, dash='dot'),
    text=["", f"y={point[1]}"],
    textposition="middle right",
    name='Y Projection'
))

# Perpendicular lines from the point to the axes
fig.add_trace(go.Scatter(
    x=[point[0], point[0]], y=[0, point[1]],
    mode='lines',
    line=dict(color='gray', width=1, dash='dot'),
    name='Coordinates'
))

# Configure the figure
fig.update_layout(
    title="Reference Frame at Origin and Point P",
    xaxis=dict(range=[-2, 2], title="X"),
    yaxis=dict(range=[-2, 2], title="Y"),
    width=600, height=600,
    plot_bgcolor='white',
    yaxis_scaleanchor="x",  # Ensures that X and Y scales are equal
    yaxis_scaleratio=1
)

# Grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True, zerolinewidth=2, zerolinecolor='gray')

fig.show()